In [1]:
import pandas as pd
import datetime
import math

In [2]:
def date_parser(strs):
    ans = []
    for s in strs:
        t = datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
        minute = int(math.floor(t.minute / 20) * 20)
        t = datetime.datetime(t.year, t.month, t.day, t.hour, minute, 0)
        ans.append(t)
    return ans

df_vol = pd.read_csv('../dataSets/training/volume(table 6)_training.csv', parse_dates=[0], date_parser=date_parser)
df_vol.sort_values(by=['time'], inplace=True)

In [3]:
# Replace empty vehicle_type with mean value

mean = df_vol['vehicle_type'].mean()
df_vol['vehicle_type'].fillna(mean, inplace=True)
df_vol.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
333995,2016-09-19,1,1,1,0,0.0
334129,2016-09-19,3,1,1,1,0.0
334125,2016-09-19,3,1,1,1,0.0
334121,2016-09-19,3,1,1,0,0.0
334120,2016-09-19,3,1,1,0,1.0


In [4]:
df_vol.describe()

,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
count,543699.000000,543699.000000,543699.000000,543699.000000,543699.000000
mean,2.086138,0.391227,1.089689,0.221420,0.225970
std,0.925612,0.488026,0.838314,0.415203,0.261589
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,0.000000
50%,2.000000,0.000000,1.000000,0.000000,0.225970
75%,3.000000,1.000000,1.000000,0.000000,0.225970
max,3.000000,1.000000,7.000000,1.000000,1.000000


In [5]:
# Transform datetime to date, weekday, time
def split_datetime(df):
    df.rename(index=str, columns={'time': 'datetime'}, inplace=True)

    df_date = df['datetime'].apply(lambda x: x.date()).to_frame()
    # df_week = df_vol['datetime'].apply(lambda x: x.weekday()).to_frame()
    df_time = df['datetime'].apply(lambda x: x.time()).to_frame()

    df_date.rename(index=str, columns={'datetime': 'date'}, inplace=True)
    # df_week.rename(index=str, columns={'datetime': 'weekday'}, inplace=True)
    df_time.rename(index=str, columns={'datetime': 'time'}, inplace=True)

    # df_vol = pd.concat([df_date, df_week, df_time, df_vol], axis=1)
    df = pd.concat([df_date, df_time, df], axis=1)
    df.drop('datetime', axis=1, inplace=True)
    df_date = None
    # df_week = None
    df_time = None
    return df
    
df_vol = split_datetime(df_vol)
df_vol.head()

,date,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
333995,2016-09-19,00:00:00,1,1,1,0,0.0
334129,2016-09-19,00:00:00,3,1,1,1,0.0
334125,2016-09-19,00:00:00,3,1,1,1,0.0
334121,2016-09-19,00:00:00,3,1,1,0,0.0
334120,2016-09-19,00:00:00,3,1,1,0,1.0


In [6]:
# Cut the specified dates in [begin, end]
def date_interval(df, begin, end):
    date_begin = datetime.datetime.strptime(begin, '%Y-%m-%d').date()
    date_end = datetime.datetime.strptime(end, '%Y-%m-%d').date()
    mask = (df['date'] >= date_begin) & (df['date'] <= date_end)
    return df[mask]

df_vol = date_interval(df_vol, '2016-09-20', '2016-09-26')
df_vol.head()

,date,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
343873,2016-09-20,00:00:00,3,1,1,0,0.0
343885,2016-09-20,00:00:00,3,1,1,1,0.0
343884,2016-09-20,00:00:00,3,1,1,0,0.0
343883,2016-09-20,00:00:00,3,1,1,0,0.0
343882,2016-09-20,00:00:00,3,1,1,0,0.0


In [7]:
# As its name suggests
def get_features(df):
    df_count = df.groupby(['date', 'time', 'tollgate_id', 'direction']).count()
    df_count = df_count['vehicle_model'].to_frame()
    df_count.rename(index=str, columns={'vehicle_model': 'count'}, inplace=True)

    df_mean = df.groupby(['date', 'time', 'tollgate_id', 'direction']).mean()
    columns = {
        'vehicle_model': 'model_mean',
        'has_etc': 'etc_mean',
        'vehicle_type': 'type_mean'
    }
    df_mean.rename(index=str, columns=columns, inplace=True)

    df_sum = df.groupby(['date', 'time', 'tollgate_id', 'direction']).sum()
    columns = {
        'vehicle_model': 'model_sum',
        'has_etc': 'etc_sum',
        'vehicle_type': 'type_sum'
    }
    df_sum.rename(index=str, columns=columns, inplace=True)

    df_feat = pd.concat([df_count, df_mean, df_sum], axis=1)
    df_count = None
    df_mean = None
    df_sum = None
    return df_feat

df_vol = get_features(df_vol)
df_vol.head()

count  model_mean  etc_mean  \
date       time     tollgate_id direction                                
2016-09-20 00:00:00 1           0             14    1.571429  0.071429   
                                1            139    1.482014  0.179856   
                    3           0             17    1.117647  0.117647   
                                1            231    1.168831  0.255411   
           00:20:00 1           0             13    1.769231  0.000000   

                                           type_mean  model_sum  etc_sum  \
date       time     tollgate_id direction                                  
2016-09-20 00:00:00 1           0           0.225970         22        1   
                                1           0.273381        206       25   
                    3           0           0.225970         19        2   
                                1           0.090909        270       59   
           00:20:00 1           0           0.225970         23        0   

                                            type_sum  
date       time     tollgate_id direction             
2016-09-20 00:00:00 1           0           3.163575  
                                1          38.000000  
                    3           0           3.841484  
                                1          21.000000  
           00:20:00 1           0           2.937605